In [4]:
import json

# import datasets
#known unknown dataset
logic_fp = 'BIG-bench/bigbench/benchmark_tasks/logic_grid_puzzle/task.json'
with open(logic_fp, 'r') as json_file:
    data = json.load(json_file)



In [5]:
for q in data["examples"][:1]:
    print(q)


{'input': 'There are 5 houses in a row, numbered 1 on the left to 5 on the right. There is one person living in each house. The people in these houses have different characteristics:\n - Each person ate something different for lunch: one ate fried rice, one ate spaghetti, one had pizza, one had stew, and one ate grilled cheese\n - Each person has a different device: one has a phone, one has a television, one has a gameboy, one has a radio, and one has a computer\n - Each person plays a different sport: one is a basketball player, one is a tennis player, one is a swimmer, one is a baseball player, and one is a soccer player\n - Each person is wearing different shoes: one is wearing high heels, one is wearing loafers, one is wearing boots, one is wearing running shoes, and one is wearing sandals\n - Each person has a favorite drink: one only drinks water, one likes milk, one is a tea drinker, one is a coffee drinker, and one is a root beer lover\n\nClue(s):\n1. The person who has a compu

In [6]:
for q in data["examples"][:3]:
    input_text = q["input"]
    target_text = q["target_scores"]
    print(input_text)
    print(target_text)
    

There are 5 houses in a row, numbered 1 on the left to 5 on the right. There is one person living in each house. The people in these houses have different characteristics:
 - Each person ate something different for lunch: one ate fried rice, one ate spaghetti, one had pizza, one had stew, and one ate grilled cheese
 - Each person has a different device: one has a phone, one has a television, one has a gameboy, one has a radio, and one has a computer
 - Each person plays a different sport: one is a basketball player, one is a tennis player, one is a swimmer, one is a baseball player, and one is a soccer player
 - Each person is wearing different shoes: one is wearing high heels, one is wearing loafers, one is wearing boots, one is wearing running shoes, and one is wearing sandals
 - Each person has a favorite drink: one only drinks water, one likes milk, one is a tea drinker, one is a coffee drinker, and one is a root beer lover

Clue(s):
1. The person who has a computer lives somewhere

In [7]:
#input text
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

# Download punkt pre-trained dataset for nlp tasks (word seg, text split)
nltk.download('punkt')

from sklearn.pipeline import Pipeline
from nltk.stem import PorterStemmer
from sklearn.preprocessing import FunctionTransformer


# preprocessing pipeline: removing stopwords, stemming, or applying custom text-based transformations
def stem_text(text):
    stemmer = PorterStemmer()
    return ' '.join([stemmer.stem(word) for word in word_tokens])

# Create a preprocessing pipeline
text_preprocessor = Pipeline([
    ('stemming', FunctionTransformer(func=stem_text, validate=False))
])

# Contain all the semantic entries for input data 
preprocessed_list = []
truth_list = []


# word tokenization 
for q in data["examples"]:
    input_text = q["input"]
    target_text = q["target_scores"]
    word_tokens = word_tokenize(input_text)

    # Apply the preprocessing pipeline to  tokenized text data. Contains the result of applying text-based transformation after stemming is applied to each token 
    preprocessed_text = text_preprocessor.fit_transform(input_text)

    #append the tokens of preprocessed text to the lists
    preprocessed_list.append(preprocessed_text)
    truth_list.append(target_text)


# generate preprocessed dataset for train 
len(preprocessed_list)

[nltk_data] Downloading package punkt to /Users/tevykuch/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


1000

In [8]:

import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer

vector_size = 100  # Dimensionality of word embeddings
window_size = 5    # Context window size
min_count = 0.5     # Minimum word frequency
sg = 0              # Use Skip-gram model (0 for CBOW)

#create a word2vec model
word2vec_model = Word2Vec(sentences = preprocessed_list, vector_size=vector_size, window=window_size, min_count=min_count, sg=sg)


# Save the trained model
word2vec_model.save('word2vec_model.bin')


text_vectors = []
for sentence in preprocessed_list:
    # Get the word vectors for each word in the sentence
    word_vectors = [word2vec_model.wv[word] for word in sentence]
    
    # Calculate the sentence vector as the mean of word vectors
    sentence_vector = np.mean(word_vectors, axis=0)
    
    text_vectors.append(sentence_vector)


# Convert the list of text vectors into a numpy array containing numerical representations
#text_vectors = np.array(text_vectors)
text_vectors[0]

array([-2.38868147e-01,  1.57130122e-01, -1.98657185e-01,  7.07081780e-02,
        1.60884112e-02, -1.28634304e-01, -6.09700382e-02,  9.46452096e-02,
        3.32956463e-02, -1.95430562e-01,  7.83713758e-02, -3.62142213e-02,
        6.06287606e-02, -1.21323243e-01,  9.07008648e-02,  5.43111898e-02,
        4.88120057e-02,  2.18408316e-01, -4.52591479e-02, -2.54116178e-01,
        1.73143402e-01,  2.36945949e-03,  8.74690339e-03,  7.11226836e-02,
        1.50570184e-01, -1.93555113e-02,  1.85503811e-01,  7.55094811e-02,
        2.07841918e-02,  1.64778113e-01, -3.71654272e-01,  2.85769105e-01,
        1.64738074e-01,  1.00546665e-01,  6.82918578e-02,  3.56733024e-01,
        5.11688553e-02, -1.63595363e-01, -2.77839571e-01,  4.02103603e-01,
        1.84386790e-01,  3.59024368e-02, -4.81859175e-03, -1.60393193e-01,
       -3.16908918e-02, -7.01410472e-02,  2.44593918e-01, -2.08912700e-01,
       -1.25493035e-02,  1.73319108e-03,  1.59540892e-01, -7.04935715e-02,
       -2.82901525e-01,  

In [9]:

# test the model with sample word 
#word_vector = word2vec_model.wv['soccer']

preprocessed_list[0]

# check if the vocabulary exists 
vocabulary = word2vec_model.wv.index_to_key
print(vocabulary)



[' ', 'e', 'o', 'h', 't', 'i', 'r', 'n', 'a', 's', 'l', 'f', 'w', 'd', 'p', 'u', 'c', '.', 'v', 'b', 'k', ',', 'm', 'g', ':', '1', 'y', '-', '2', '3', '4', 'q', '5', 'x', ')', '(', '?', 'z', '6', '7', '8', '9', '0']


In [10]:
# check if preprocessed words in the model 
if 'soccer' in word2vec_model.wv:
    soccer_vector = word2vec_model.wv['soccer']
    print(soccer_vector)
else:
    print('soccer not in vocabulary')


soccer not in vocabulary


In [11]:
contains_soccer = any('soccer' in token for token in preprocessed_list)
if contains_soccer:
    print('soccer is in the list')
else:
    print('soccer is not in the list')


#preprocessed_list is a list of list 


soccer is in the list


In [12]:
for sentence_tokens in preprocessed_list:
    if 'soccer' in sentence_tokens:
        print('soccer is in the list')
        break


soccer is in the list


In [13]:
import pandas as pd

input_df = pd.DataFrame({'input text' : preprocessed_list})
truth_df = pd.DataFrame({'truth text' : truth_list})

combined_df = {'input text' : input_text,
'tokenized text' : text_vectors,
'truth text' : truth_list }

combineddf = pd.DataFrame({'combined text' : combined_df})
combineddf[:3]

,combined text
input text,"There are 2 houses next to each other, numbere..."
tokenized text,"[[-0.23886815, 0.15713012, -0.19865718, 0.0707..."
truth text,"[{'1': 0, '2': 0, '3': 0, '4': 1, '5': 0}, {'1..."


In [22]:
combined = pd.DataFrame(combined_df)
combined[:3]

,input text,tokenized text,truth text
0,"There are 2 houses next to each other, numbere...","[-0.23886815, 0.15713012, -0.19865718, 0.07070...","{'1': 0, '2': 0, '3': 0, '4': 1, '5': 0}"
1,"There are 2 houses next to each other, numbere...","[-0.2377491, 0.15154833, -0.19432862, 0.080036...","{'1': 0, '2': 0, '3': 1, '4': 0}"
2,"There are 2 houses next to each other, numbere...","[-0.25480124, 0.14128003, -0.22882473, 0.04912...","{'1': 0, '2': 0, '3': 1, '4': 0}"


In [23]:
correct_labels = {}
#truth list is a list of dictionaries
for labels in truth_list: 
    for label, score in labels.items(): 
        if score == 1:
            correct_labels[label] = score

In [25]:
reward = [{
    'input text': input_text,
    'label' : label,
    'score' : score
} for label, score in correct_labels.items()]

reward_mod = pd.DataFrame(reward)
reward_mod[:4]


,input text,label,score
0,"There are 2 houses next to each other, numbere...",4,1
1,"There are 2 houses next to each other, numbere...",3,1
2,"There are 2 houses next to each other, numbere...",2,1
3,"There are 2 houses next to each other, numbere...",1,1


## Preprocessing Architecture

In [97]:
# Preprocessing Setup 
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import torchvision.models as models
import torch.utils.data as data
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

# split data into train and test sets randomly by percentage weight using the dataframe 
train_data, test_val_data = train_test_split(combined, test_size=0.1, random_state=42)

#split test into test and val by percentage weight
test_data, val_data = train_test_split(test_val_data, test_size=0.5, random_state=42)

batch_size = 16

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size, shuffle=True)


### output layer for preprocessing

In [98]:
# use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Multiple Output Layers ## 
# Loss Function: Binary Cross-Entropy Loss: binary classification output layer
# Mean Squared Error (MSE) or Mean Absolute Error (MAE) : Regression layer to predict a continuous confidence score

class CustomOutputLayer(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(CustomModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3_binary = nn.Linear(hidden_size, 1)  # Binary classification output layer
        self.fc3_regression = nn.Linear(hidden_size, 1)  # Regression output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        binary_output = torch.sigmoid(self.fc3_binary(x))  # Apply sigmoid for binary classification
        regression_output = self.fc3_regression(x)  # Linear activation for regression
        return binary_output, regression_output


    #training epoch, loss function